# Lab 1: R and Tidyverse Basics

## Working with R Scripts, Folders, and Files

Unlike most computing that you may be used to in this age of cloud services and data centers, we will be storing our files locally on your personal computer (or using the campus computer lab if needed). That means we need to tell R where our necessary files and folders are located. It won't matter too much for most of our labs, but when you work on your own projects, you will need to become profecient with accessing many files and folders on your computer. In R and RStudio, there are two ways of doing this. First, you can set up an RStudio Project. 

Steps:

- File &rarr; New Project
- Choose a name for the project and folder location.^[I have a folder called "labs" located in the my main PUBG 511 folder. If you are primarily going to be using RStudio through the computer lab or the Virtual App, then place this folder in your H:\ network space]
- Now start a new script using the sheet with a plus sign icon from the toolbar or using the file menu. On the right side of your window, it should show the name of your project.
- RStudio projects make it so you don't need to worry about setting a working directory - it is defined in the project. Just make sure all files written to or read by R are in the same folder or in a nested subfolder. 

Second, in any R script or R session, you can specify the working directory. The working directory is the location on your computer that R will search for files or folders. So, for example, I could tell R to set my working directory to "/home/dan/Courses/PUBG_511/labs" and then any file I refer to in the script can be referenced in relation to my /labs folder without me specifying the rest of the file path. 

The syntax is:

```r 
setwd("filepath")
```

So, for my work, I might say:

```r
setwd("/home/dan/Courses/PUBG_511/labs")
```
and then to import a .csv file located in the `labs` folder, I could simply code:

```r
data <- read_csv("./some_data_set.csv")
```

Note that how your computer refers to files and folders differs depending on your operating system. On Unix-based systems (macOS, Linux), folders are organized by user like: "/myname/" or "/home/name/", and folders are separated by forward slashes (/). On Windows, files have drive names like "C:\" and then folders are separated by backslashes (\). **Note that R does not accept the backslash in filepaths. So you either need to replace the backslash with a forward slash or use two backslashes (\\) to include the filepath. 

## Installing and loading packages

R is open-source, and, frankly, kinda stinks on its own.^[This is called base R. ] But there are many, many user-generated packages that improve R's functionality. We'll be using these packages all the time, especially a group of packages called [the tidyverse](https://www.tidyverse.org/packages/). 

You only need to install the package once and then you're good to go (until it needs updating). But you also need to load the package in every R session if you want to use those commands. 

In [ ]:
# Install the required packages if not already installed 
# By the way, the hashtag/pound/octothorpe symbol will comment out a line in your script

install.packages(c('tidyverse', 'dataverse', 'modelsummary'))

# let's load your packages in the R session

library(tidyverse)
library(dataverse)
library(modelsummary)

## Some basic built-in functions

R can handle a great diversity of *objects* including lists, variables, names, vectors, data frames, scalars, and plots. Let's create a vector of data using a random draw from a normal distribution and then use two functions to describe the variable.

In [ ]:
x <- rnorm(2500, mean = 50, sd = 10) # assigns object `x` to receive values drawn normal dist
                                     # with mean of 50 and standard deviation of 10

                                     # the `<-` reads "gets" or "is assigned"

mean(x)
summary(x)

We can check the distribution of our variable using a histogram.

In [ ]:
hist(x)

We could change the number of bins used by the histogram if we want and change the y-axis from a frequency to a probability (much better). R options will typically require a value (like the 30 in the breaks option below) or a logical entry of TRUE or FALSE to toggle settings on or off.

In [ ]:
hist(x, breaks = 30, probability = TRUE)

Let's run hist() once more, this time suppressing the data and asking R to give use the output in the console.

In [ ]:
hist(x, breaks = 30, probability = TRUE, plot = FALSE)

## Opening datasets

R has various formats for datasets, typically called a data frame. for this lab, we will be using a dataset created by Meier, Johnson, and An for their paper "Perceptual Bias and Public Programs: The Case of the United States and Hospital Care." The dataset is part of the replication file their [PAR paper](https://onlinelibrary.wiley.com/doi/full/10.1111/puar.13067). In the paper, the authors are retesting a previous finding that citizens had negative opinions about public services, compared to private services. Respondents were randomly assigned to receiving a cue about a public or private hospital, and then asked to evaluate the organization on several items. We'll use one: *q9*: "The hospital acts in the interest of patients", scored on a scale from 1 (doesn't fit) to 7 (fits very well). Dataverse repositories allow files to be downloaded in multiple formats: usually as a Stata file, an R workspace, or as a .csv or .tab delimited text file. If you were to place your files in the same folder that you have set as your project location or working directory, you would not need to specify working directories or complete file paths; your computer will already know were to look for the data.

We can load an Rdata workspace using the `load()` function. We can also import datasets saved in other formats like .csv or .dta files using various R packages. To open the Stata .dta file, here stored in JOP dataverse, we'll need to use functionality from the `read_dta()` function from the **haven** package. But instead, let's use the functionality of the **dataverse** package to directly ingest the data from the online repository in to R. 

In [ ]:
public.hosp <- get_dataframe_by_doi(                 # we'll name the dataframe public.hosp
          filedoi = "doi:10.7910/DVN/NRQPZX/KOEWTQ", # this is the code for the specific table we want
          original = TRUE,                           # original dataset is Stata 13 binary file
          .f = haven::read_dta,                      # to read Stata file, use read_dta() function
          server = "dataverse.harvard.edu")          # the paper's files are stored on this server

head(public.hosp)

## Using the pipe (|>)

The pipe operator allows users to pass objects onto multiple functions without creating new variables or nesting functions. The original pipe is built in to the **tidyverse** packages (%>%) - this is known as the **magrittr** pipe, and you'll see many example codes online using it. It's great. However, starting in R version 4.1, base R includes a new pipe operator that will work through many R functions (`|>`). Let's look at the example below using **tidyverse**'s *summarize* function, which aggregates data in the ways specfied in the command. In this case, we're summarizing two variables (respondent age and respondent self-reported ideology) by asking R to calculate their means. 


In [ ]:
public.hosp |> summarize(mean_age = mean(age, na.rm = TRUE), 
                         mean_ideo = mean(ideology, na.rm = TRUE))

So here, the pipe passes the data (public.hosp) to the summarize function. We could have specified the data directly inside the summarize function if we wanted to. But using the pipe will let us have a great deal more control over the how we process the data. 

## **filter** and **group_by**

These functions are also part of the **tidyverse**. `filter()` allows us to, well, filter or subset our observations based on some condition or a list of conditions. In the code below, we just respondents who are women. 

The `group_by()` function allows you to aggregate your data by grouping variables. In the code below, we group the data by race variable (1 is white and 0 non-white). 

In [ ]:
public.hosp |> filter(gender == 1) |>       # use double equals signs for equality when you are writing 
       group_by(white) |>    #     conditional statements
                summarize(mean_age = mean(age, na.rm = TRUE), 
                          mean_ideo = mean(ideology, na.rm = TRUE))

The pipe first passes the data to be filtered, then passes the filtered dataset to be grouped, then passes the grouped and final data to be summarized. If you want to insert the pipe using a keyboard shortcut in RStudio, you can use <kbd>ctrl</kbd> + <kbd>shift</kbd> + <kbd>m</kbd>. Note that the shortcut uses the **tidyverse** pipe by default, but you can change it to the base R |> pipe in the settings. Check the "Use native pipe operator, |> (requires $ 4.1+) box in Tools &rarr; Global Options &rarr; Code menu window.

## Creating a table of descriptive statistics 

To create a table of descriptive statistics, we will use the `datasummary_skim()` function of the **modelsummary** package. This is a clean, well-formatted table output that will give us a nice set of options for exporting, viewing, or saving the table. The `select()` function is part of **tidyverse** and will allow us to filter out some columns of data if we want. Check out the [package description](https://modelsummary.com/vignettes/datasummary.html) for further functions and uses for `datasummary`

In [ ]:
datasummary_skim(public.hosp)


# here we write our table to a new stored data frame called desc.tab.out
desc.tab.out <- public.hosp |> 
    select(!(c("q1", "q2"))) |>    # ! means "does not", so we are selecting
    datasummary_skim()             # just those columns that don't equal q1 or q2.
                                   # the c() stands for "concatenate" - it lets you
                                   #    link items together. 
                                   # We don't need to include the name of the dataframe
                                   # in the final line because it is being piped in

# type the name to view the stored object
desc.tab.out

We can also choose which statistics we want calculated, as well as specific variables we would like included in the table:

In [ ]:
# Here we list the variables in the left side of the formula, 
# and then the statistics on the right.
datasummary(data = public.hosp,
            formula = q3 + q4 + public + performance + age ~ Mean + Median + SD + Histogram)

## Recoding and generating new variables

Changing variables and creating new variables can be done in both base R and using the tidyverse. I typically use the **tidyverse** function `mutate()` for recoding and generating new variables. To illustrate how `mutate` works, let's recode the age variable in the dataset. Notice a couple of items about the following code. First, if we want the changes to remain, we need to assign the output of the function to a new object, or overwrite the existing object by assigning it its same. I would strongly recommend assigning to a new object when recoding, like in the code below. Second, you can string together multiple calls to recode or create new variables inside the same `mutate` call, just string them together with a comma. Finally, to recode from interval data into categories, we'll use the `case_when()` function. This function will build, so it matters which order you list the conditions. If you list something last that contradicts what you said earlier, then the earlier recode will be overwritten. The final line of the `case_when` statement takes all of the remaining values and codes them to missing data.

If we want to label the values of our new variable, we can use R's `ordered()` function, which tells R the variable is ordinal and allows us to add labels. 

In [ ]:
# here we write the recoded dataframe to a new dataframe, otherwise your changes will be
#   printed in the console but not stored.
#   
public.hosp.recode <- public.hosp |> 
                            mutate(age.squared = age^2, # you can perform mathematical operations on variables
                                   age4cat = case_when(
                                        age < 35 ~ 0,
                                        age >= 35 & age < 50 ~ 1,
                                        age >= 50 & age < 65 ~ 2,
                                        age >= 65 ~ 3,
                                        TRUE ~ NA_real_ # anything not already assigned gets NA
                                   ))

public.hosp.recode$age4cat <- ordered(public.hosp.recode$age4cat,
                                   labels = c("0-34", "34-49", "50-64", "65+"))

# quick counts of observation for each value
table(public.hosp.recode$age4cat)

# let's wrap in prop.table() function to get proportions:
prop.table(table(public.hosp.recode$age4cat))

## Crosstabs 

I use cross-tabulations, or contingency tables, quite a bit in my own work. They are simple ways of seeing the joint frequency distribution of two (or more) variables together. There are several packages in R that can create crosstabs. Instead of using a specially-design package first, let's rely on the base R functions of `table()` and `prop.table`.

In [ ]:
CT.summary <- table(public.hosp$q9, public.hosp$public)
CT.summary

# notice that we can wrap CT.summary inside of prop.table to get
# proportions. the , 2 specifies that we want column proportions
# We also wrap that output in round() to round our results.
CT.final <- round(prop.table(CT.summary, 2) * 100, digits = 1)
CT.final

Crosstabs are even easier using `datasummary_crosstab()`, again from the **modelsummary** package.

In [ ]:
datasummary_crosstab(q9 ~ public,
                     statistic = ~ 1 + N + Percent("col") + Percent("row"),
                     data = public.hosp.recode)

These tables would be better if labeled the values of our variables.

In [ ]:
# first, create a list of value labels, in order

pub.lbl <- c("No Public Cue", "Public Cue")
q9.lbl <- c("Not at All", "2", "3", "Neither", "5", "6", "Very Well")

# now create new variables, telling R that we have factor and ordinal data 
# and attaching the lists as variable labels
public.hosp.recode <- public.hosp.recode |>
                        mutate(public_fct = factor(public,labels = pub.lbl),
                               q9_ord = ordered(q9, labels = q9.lbl))

#recreate crosstab using labelled data
datasummary_crosstab(q9_ord ~ public_fct,
                     statistic = ~ 1 + N + Percent("col") + Percent("row"),
                     data = public.hosp.recode)

## Mean comparisons and difference of means tests

Mean comparisons are easier, and we can calculate the mean comparison test simply by grouping our data and then running `summarise()` with the `mean()` function. Grouping is done by the independent variable in a mean comparison test, we are calculating the average of the dependent variable.

In [ ]:
MC.tab <- public.hosp.recode |>
                    group_by(public) |>
                    summarise("Mean of q9" = mean(q9))

MC.tab

Hmmm. These look pretty similar. I'm not sure there is much of an effect of the public hospital prompt in the survey experiment. Let's test it more formally by calculating a t-test, which evaluates whether $\bar{y_{private}} - \bar{y_{public}}$ is significantly different from the null hypothesis of 0. 

In [ ]:
t.test(q9 ~ public, data = public.hosp.recode, var.equal = TRUE)